In [1]:
import numpy as np
import scipy.spatial.distance as sci_dist
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import pandas as pd
import numpy as np

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

from wordcloud import WordCloud

In [2]:
def read_input(dataset_file_path):
    #arq = open(dataset_file_path, 'r')
    #doc = arq.readlines()
    #arq.close()

    #documents = list(map(str.rstrip, doc))
    #n_documents = len(documents)
    
    %time 
    df = pd.read_pickle(dataset_file_path, compression='xz')
    print('{} docs shape'.format(df.shape))
    documents = df['clean'].values.tolist()
    string = ''
    for index, document in enumerate(documents):
        for word in document:
            string += word + ' '
        string = string[:-1]
        documents[index] = string 
        string = ''
    n_documents = len(documents)
    return documents, n_documents
documents, n_documents = read_input('2017_lattes.pkl.xz')
#documents, n_documents = read_input('2017_lattesPre.txt')
documents[0]

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.78 µs
(8652, 4) docs shape


'exploit photo locat direct cluster base point interest discoveri'

In [3]:
def count(docPre):
        # print docPre
        count_vec = CountVectorizer(binary=True)  # , min_df=0.0,max_df=1.0, token_pattern=r"(?u)\b\w+\b")
        count = count_vec.fit_transform(docPre)
        words = list(map(str, count_vec.get_feature_names()))

        # Sem print words nao funciona kkk
        # print words
        # input("ok")
        # exit()

        n_terms = len(words)

        count_t = count.transpose()

        word_frequency = {}
        term_docs = {}

        # print type(count)
        # trocar essa parte (change)
        for i in range(n_terms):
            word_frequency[words[i]] = float(count_t[i].getnnz(1))
            term_docs[words[i]] = set(count_t[i].nonzero()[1])

        # print term_docs

        # word_frequency['mm'] = word_frequency['tum']
        # term_docs['mm'] = term_docs['tum']

        # print word_frequency["video"]
        # print word_frequency["call"]

        return n_terms, words, word_frequency, term_docs
n_terms, words, word_frequency, term_docs = count(documents)

In [4]:
def get_topics(topics_path):
    arq = open(topics_path, 'r')
    topics = arq.readlines()
    arq.close()
    return topics
topics = get_topics('LDA_Topics')

In [5]:
def coherence(topic, word_frequency, term_docs):
    coherence = []

    for t in range(len(topic)):
        topico = topic[t]
        top_w = topico.split(" ")

        coherence_t = 0.0
        for i in range(1, len(top_w)):
            for j in range(0, i):
                cont_wi = word_frequency[top_w[j]]
                cont_wi_wj = float(
                    len(term_docs[top_w[j]].intersection(term_docs[top_w[i]])))
                coherence_t += np.log((cont_wi_wj + 1.0) / cont_wi)

        coherence.append(coherence_t)

    return coherence

In [6]:
def pmi(topics, word_frequency, term_docs, n_docs, n_top_words):
    pmi = []
    npmi = []

    n_top_words = float(n_top_words)

    for t in range(len(topics)):
        top_w = topics[t]
        # top_w = topico.split(' ')

        pmi_t = 0.0
        npmi_t = 0.0

        for j in range(1, len(top_w)):
            for i in range(0, j):
                ti = top_w[i]
                tj = top_w[j]

                c_i = word_frequency[ti]
                c_j = word_frequency[tj]
                c_i_and_j = len(term_docs[ti].intersection(term_docs[tj]))

                pmi_t += np.log(((c_i_and_j + 1.0) / float(n_docs)) /
                                ((c_i * c_j) / float(n_docs) ** 2))

                npmi_t += -1.0 * np.log((c_i_and_j + 0.01) / float(n_docs))

        peso = 1.0 / (n_top_words * (n_top_words - 1.0))

        pmi.append(peso * pmi_t)
        npmi.append(pmi_t / npmi_t)

        return pmi, npmi

In [7]:
def w2v_metric(topics, t, path_to_save_model, distance_type, dataset, embedding_type=False):
    word_vectors = KeyedVectors.load_word2vec_format(
        fname='{}/{}.txt'.format(path_to_save_model, dataset), binary=embedding_type)
    model = word_vectors.wv
    values = []

    for topic in topics:
        words = topic.split(' ')
        value = Evaluation._calc_dist_2(words, model, distance_type, t)
        values.append(value)

    return values

In [10]:
arq = open('dataset.txt', 'w')
for doc in documents:
    arq.write(doc)
arq.close()

In [9]:
t = 10
with open('{}/result_topic_{}.txt'.format('.', t), 'w') as f_res:
    f_res.write('Topics {}\n'.format(t))
    f_res.write('Topics:\n')
    topics_t = []
    for topic in topics:
        topics_t.append(topic[:t])
        for word in topic[:t]:
            f_res.write('{} '.format(word))

        f_res.write('\n')

    coherence = coherence(topics_t, word_frequency, term_docs)
    f_res.write('Coherence: {} ({})\n'.format(np.round(np.mean(coherence), 4), np.round(np.std(coherence), 4)))
    f_res.write('{}\n'.format(coherence))

    pmi, npmi = pmi(topics=topics_t,
                               word_frequency=word_frequency,
                               term_docs=term_docs,
                               n_docs=n_docs,
                               n_top_words=t)
    # f_res.write('PMI: {} ({})\n'.format(np.round(np.mean(pmi), 4), np.round(np.std(pmi), 4)))
    # f_res.write('{}\n'.format(pmi))
    f_res.write('NPMI:\n')
    for score in npmi:
        f_res.write('{}\n'.format(score))

    f_res.write('avg NPMI: {} ({})\n'.format(np.round(np.mean(npmi), 4), np.round(np.std(npmi), 4)))

    # w2v_l1 = Evaluation.w2v_metric(topics, t, path_to_save_model, 'l1_dist', dataset)
    # f_res.write('W2V-L1: {} ({})\n'.format(np.round(np.mean(w2v_l1), 4), np.round(np.std(w2v_l1), 4)))
    # f_res.write('{}\n'.format(w2v_l1))

    f_res.close()

KeyError: 'virtua'